### telemetry ts dataset  + oracle features

base: 9.DeepModels/telemetry_dataset.ipynb

Build a time series dataset across all the oval races, including telemetry: distance, vspeed(vehicle speed), espeed(engine speed), brake, throttle, etc. When this dataset aims to be used in forecasting, covariates of the racing status can not be included, such as track_status and lap_status. However, they can be used in oracle test to tell the upper bound of performance of the predictor.

+  [(eventid,carids: carno -> rowid, datalist)]

datalist := [datalist_entry] in shape of #car_number

datalist_entry := [[time_seq_id, distance, vspeed, track_status, lap_status]], in shape of #ts_len x #featureCnt

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import mxnet as mx
from mxnet import gluon
import pickle
import json
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

## Load Data

In [2]:
import os
os.getcwd()

'/scratch/hpda/indycar/notebook/10.OracleModel'

In [3]:
#
# parameters
#
#year = '2017'
year = '2018'
#event = 'Toronto'
#https://www.racing-reference.info/season-stats/2018/O/#
events_totalmiles=[256,500,372,268,500,310]
events_laplen = [1.022,2.5,1.5,0.894,2.5,1.25]
events = ['Phoenix','Indy500','Texas','Iowa','Pocono','Gateway']
#events = ['Gateway']

#events = ['Indy500']
#events = ['Phoenix']
events_id={key:idx for idx, key in enumerate(events)}

In [4]:
# make indy car completed_laps dataset
# car_number, completed_laps, rank, elapsed_time, rank_diff, elapsed_time_diff 
def make_cl_data(dataset):

    # pick up data with valid rank
    rankdata = dataset.rename_axis('MyIdx').sort_values(by=['elapsed_time','MyIdx'], ascending=True)
    rankdata = rankdata.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')

    # resort by car_number, lap
    uni_ds = rankdata.sort_values(by=['car_number', 'completed_laps', 'elapsed_time'], ascending=True)    
    uni_ds = uni_ds.drop(["unique_id", "best_lap", "current_status", "track_status", "lap_status",
                      "laps_behind_leade","laps_behind_prec","overall_rank","pit_stop_count",
                      "last_pitted_lap","start_position","laps_led"], axis=1)
    
    carnumber = set(uni_ds['car_number'])
    print('cars:', carnumber)
    print('#cars=', len(carnumber))
   
    # faster solution , uni_ds already sorted by car_number and lap
    uni_ds['rank_diff'] = uni_ds['rank'].diff()
    mask = uni_ds.car_number != uni_ds.car_number.shift(1)
    uni_ds['rank_diff'][mask] = 0
    
    uni_ds['time_diff'] = uni_ds['elapsed_time'].diff()
    mask = uni_ds.car_number != uni_ds.car_number.shift(1)
    uni_ds['time_diff'][mask] = 0
    
    df = uni_ds[['car_number','completed_laps','rank','elapsed_time','rank_diff','time_diff']]
    
    return df

def make_lapstatus_data(dataset):
    final_lap = max(dataset.completed_laps)
    total_laps = final_lap + 1

    # get records for the cars that finish the race
    completed_car_numbers= dataset[dataset.completed_laps == final_lap].car_number.values
    completed_car_count = len(completed_car_numbers)

    print('count of completed cars:', completed_car_count)
    print('completed cars:', completed_car_numbers)
    
    #pick up one of them
    onecar = dataset[dataset['car_number']==completed_car_numbers[0]]
    onecar = onecar.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')
    return onecar[['completed_laps','track_status']]
    

In [5]:
def load_data(event, year):
    inputfile = '../data/final/C_'+ event +'-' + year + '-final.csv'
    outputprefix = year +'-' + event + '-'
    dataset = pd.read_csv(inputfile)
    #dataset.info(verbose=True)    
    
    final_lap = max(dataset.completed_laps)
    total_laps = final_lap + 1

    # get records for the cars that finish the race
    completed_car_numbers= dataset[dataset.completed_laps == final_lap].car_number.values
    completed_car_count = len(completed_car_numbers)

    print('count of completed cars:', completed_car_count)
    print('completed cars:', completed_car_numbers)

    #make a copy
    alldata = dataset.copy()
    dataset = dataset[dataset['car_number'].isin(completed_car_numbers)]
    rankdata = alldata.rename_axis('MyIdx').sort_values(by=['elapsed_time','MyIdx'], ascending=True)
    rankdata = rankdata.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')
    
    cldata = make_cl_data(dataset)
    flagdata = make_lapstatus_data(dataset)
    acldata = make_cl_data(alldata)

    return alldata, rankdata, acldata, flagdata

### overall view of laptime scatter plots



In [6]:
def get_cardata(curcarno, ycol='time_diff'):
    car = acldata[acldata['car_number']==curcarno]
    #print(car['time_diff'].describe())
    
    cols=['completed_laps','rank','car_number','lap_status','track_status',
    'pit_stop_count','current_status','start_position']
    colid={key:idx for idx, key in enumerate(cols)}

    cardata = rankdata[rankdata['car_number'] == curcarno]

    carstatus = [[row[0], row[1],row[2],row[3],row[4],row[5],row[6],row[7]] for row in cardata[
        ['completed_laps','rank','car_number','lap_status','track_status',
        'pit_stop_count','current_status','start_position']].values]
    
    x = car['completed_laps'][1:].values
    y = car[ycol][1:].values

    pits=[]
    yellowflags=[]
    lastflag = 'x'
    for row in carstatus:
        lap = int(row[colid['completed_laps']])

        if row[colid['lap_status']]=='P':
            pits.append(lap)

        if row[colid['track_status']]=='Y':
            if lastflag != 'Y':       
                #start
                yellowflags.append(lap)
        else:
            if lastflag == 'Y':       
                #end
                yellowflags.append(lap)        
        lastflag = row[colid['track_status']]

    #pit lap
    pits = np.array(pits)
    #start, end lap
    #
    yellowflags = np.array(yellowflags)
    if (yellowflags.shape[0] % 2)==1:
        print('crash?:carno=', curcarno)
        yellowflags = []
    else:
        yellowflags = np.array(yellowflags).reshape((-1,2))    
    
    return car, x, y, pits, yellowflags

### bulid the dataset

In [7]:
stagedata = {}
global_carids = {}
traindata = None
cur_carid = 0
for event in events:
    #alldata, rankdata, acldata, flagdata
    stagedata[event] = load_data(event, year)
    
    alldata, rankdata, acldata, flagdata = stagedata[event]
    carlist = set(acldata['car_number'])
    laplist = set(acldata['completed_laps'])
    print('%s: carno=%d, lapnum=%d'%(event, len(carlist), len(laplist)))

    #build the carid map
    for car in carlist:
        if car not in global_carids:
            global_carids[car] = cur_carid
            cur_carid += 1
    

count of completed cars: 11
completed cars: [ 1  6 27  9 28  5 20 14 15 22 30]
cars: {1, 5, 6, 9, 14, 15, 20, 22, 27, 28, 30}
#cars= 11
count of completed cars: 11
completed cars: [ 1  6 27  9 28  5 20 14 15 22 30]
cars: {1, 4, 5, 6, 9, 10, 12, 14, 15, 18, 19, 20, 21, 22, 23, 26, 27, 28, 30, 32, 59, 88, 98}
#cars= 23
Phoenix: carno=23, lapnum=251
count of completed cars: 18
completed cars: [12 20  9 27 28 22 29  1  6 15 66 98  4 88 25 60 64 23]
cars: {64, 1, 66, 98, 4, 6, 9, 12, 60, 15, 20, 22, 23, 88, 25, 27, 28, 29}
#cars= 18
count of completed cars: 18
completed cars: [12 20  9 27 28 22 29  1  6 15 66 98  4 88 25 60 64 23]


/scratch/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/scratch/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


cars: {1, 3, 4, 6, 7, 9, 10, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 59, 60, 64, 66, 88, 98}
#cars= 33
Indy500: carno=33, lapnum=201
count of completed cars: 9
completed cars: [ 9 22 27  5 28 15 30 18 10]
cars: {5, 9, 10, 15, 18, 22, 27, 28, 30}
#cars= 9
count of completed cars: 9
completed cars: [ 9 22 27  5 28 15 30 18 10]
cars: {1, 3, 4, 5, 6, 7, 9, 10, 12, 14, 15, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 30, 47, 55, 57, 59, 60, 68, 73, 83, 88, 98}
#cars= 32
Texas: carno=32, lapnum=249
count of completed cars: 5
completed cars: [ 5 21 30  1  6]
cars: {1, 5, 6, 21, 30}
#cars= 5
count of completed cars: 5
completed cars: [ 5 21 30  1  6]
cars: {1, 4, 5, 6, 9, 10, 12, 14, 15, 18, 19, 20, 21, 22, 23, 26, 27, 28, 30, 59, 88, 98}
#cars= 22
Iowa: carno=22, lapnum=301
count of completed cars: 4
completed cars: [27 12  9 18]
cars: {9, 18, 27, 12}
#cars= 4
count of completed cars: 4
completed cars: [27 12  9 18]
cars: {1, 4, 5, 6, 9, 10, 12, 14, 15, 18, 

In [8]:
#alldata, rankdata, acldata, flagdata = stagedata['Indy500']
#acldata[acldata['car_number']==12].head(10)

In [9]:

def get_telemetry_dataset(stagedata, frequency = 1):
    """
    input: 
        stagedata: (alldata, rankdata, acldata, flagdata)
        frequency: 1s by default
    
    output: vspeed & distance data
    
    [(
    eventid,
    carids : rowid -> carno,
    vspeed : #car_number x variable length
    distance : #car_number x variable length
    )]
    """
    telemetry_data = []
    for event in stagedata.keys():
        eventid = events_id[event]
        
        alldata, rankdata, acldata, flagdata = stagedata[event]
        #carnumber -> carid
        carlist = set(acldata['car_number'])
        carids={key:idx for idx, key in enumerate(carlist)}
        decode_carids={idx:key for idx, key in enumerate(carlist)}
        
        # multiple ts for this event
        ts_event = []
        for carno in carlist:
            #get completed_laps
            elapsed_time = np.max(acldata[acldata['car_number'] == carno][['elapsed_time']].values)
        
            # load data for this car
            #  timestamp\t distance \t vspeed ...\t track_status \t lap_status$
            # 16:23:00.588    3705.16 153.780 10416   3       0       109.5   0.04 0 0
            inputfile = f'../data/telemetry-oracle/{event}-{year}-{carno}.csv'
            try:
                _data = pd.read_csv(inputfile,delimiter='\t', header=None)
            except:
                #Texas-car-3
                print('failed to read telemetry:', inputfile)
                ts_event.append(np.array([[0,np.nan, np.nan]]))
                continue
            
            # calc the time differences(in seconds)
            _data[0]=pd.to_datetime(_data[0])
            _data[10] = (_data[0] - _data[0].iloc[0]).dt.total_seconds()
            _data[11] = _data[10].astype(int)

            ts_length = int(elapsed_time)
            #ts = []
            features_cnt = 9
            ts = np.zeros((ts_length,features_cnt + 1))
            ts[:] = np.nan
            
            cur_id = 0
            _data_array = _data[[11,1,2,3,4,5,6,7,8,9]].to_numpy()
            last_id = _data_array.shape[0]
            
            empty_rec = [np.nan for x in range(features_cnt)]
            
            for id in range(ts_length):
                while((cur_id < last_id) and (_data_array[cur_id][0] < id)):
                    cur_id += 1

                if cur_id == last_id:
                    break
                    
                if _data_array[cur_id][0] > id:
                    #not found, missing data
                    #ts.append([id, np.nan, np.nan])
                    #ts[id,:] = [id, np.nan, np.nan]
                    _rec = [id]
                    _rec.extend(empty_rec)
                    ts[id,:] = _rec
                    
                else:
                    #ts.append([id, _data_array[cur_id][1], _data_array[cur_id][2]])            
                    #ts[id,:] = [id, _data_array[cur_id][1], _data_array[cur_id][2]]            
                    _rec = [id]
                    _rec.extend([_data_array[cur_id][1 + x] for x in range(features_cnt)])
                    ts[id,:] = _rec
            
            # deal with this ts
            # ts_event.append(np.array(ts))
            ts_event.append(ts)

        #add one record
        telemetry_data.append([eventid, decode_carids, ts_event])
        # push this event into stage dataframe
        print('event=%s, records=%s'%(event, len(telemetry_data[-1][2])))
        
        
    return telemetry_data

In [10]:
telemetry_data = get_telemetry_dataset(stagedata)

event=Phoenix, records=23
event=Indy500, records=33
failed to read telemetry: ../data/telemetry-oracle/Texas-2018-3.csv
failed to read telemetry: ../data/telemetry-oracle/Texas-2018-7.csv
failed to read telemetry: ../data/telemetry-oracle/Texas-2018-25.csv
failed to read telemetry: ../data/telemetry-oracle/Texas-2018-47.csv
failed to read telemetry: ../data/telemetry-oracle/Texas-2018-55.csv
failed to read telemetry: ../data/telemetry-oracle/Texas-2018-57.csv
failed to read telemetry: ../data/telemetry-oracle/Texas-2018-60.csv
failed to read telemetry: ../data/telemetry-oracle/Texas-2018-68.csv
failed to read telemetry: ../data/telemetry-oracle/Texas-2018-73.csv
failed to read telemetry: ../data/telemetry-oracle/Texas-2018-83.csv
event=Texas, records=32
event=Iowa, records=22
event=Pocono, records=22
event=Gateway, records=21


In [11]:
import pickle
#stintdf.to_csv('laptime-%s.csv'%year)
with open('telemetry-oracle-%s.pickle'%year, 'wb') as f:
    #pack [global_carids, laptime_data]
    savedata = [global_carids, telemetry_data]
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)

In [12]:
#test telemetry
datalist = telemetry_data[0][2]
len(datalist)

23

In [13]:
datalist[7]

array([[0.00000e+00, 1.46151e+03, 1.29290e+02, ..., 1.10000e-01,
        0.00000e+00, 0.00000e+00],
       [1.00000e+00, 1.52182e+03, 1.41940e+02, ..., 4.00000e-02,
        0.00000e+00, 0.00000e+00],
       [2.00000e+00, 1.59420e+03, 1.54320e+02, ..., 3.00000e-02,
        0.00000e+00, 0.00000e+00],
       ...,
       [6.24700e+03, 1.26581e+03, 1.49840e+02, ..., 2.40000e-01,
        0.00000e+00, 0.00000e+00],
       [6.24800e+03, 1.32697e+03, 1.54950e+02, ..., 2.30000e-01,
        0.00000e+00, 0.00000e+00],
       [6.24900e+03, 1.39738e+03, 1.60750e+02, ..., 1.90000e-01,
        0.00000e+00, 0.00000e+00]])

### convert to gluonts dataset

In [14]:

def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

def test_flag(a, bitflag):
    return (a & bitflag) ==  bitflag

#
# remove NaN at the tail
# there should be no nans in the middle of the ts
teleCols = ['seqid','distance', 'vspeed', 'rpm', 'gear', 
                 'brake', 'throttle', 'steering','track_status','lap_status']
teleColId={key:idx for idx, key in enumerate(teleCols)}

COL_DISTANCE=1
COL_VSPEED=2
COL_TRACKSTATUS=8
COL_LAPSTATUS=9

MODE_ORACLE = 0
MODE_NOLAP = 1
MODE_NOTRACK = 2
MODE_TEST = 4
MODE_STR={MODE_ORACLE:'oracle', MODE_NOLAP:'nolap',MODE_NOTRACK:'notrack',MODE_TEST:'test'}

def make_dataset(runs, prediction_length, freq, 
                       run_ts=COL_VSPEED, #teleColId['vspeed'], 
                       train_ratio = 0.8,
                       use_global_dict = True,
                       oracle_mode = MODE_ORACLE,
                       interpolate = True,
                       nan_threshold=0.1
                ):
    """
    split the ts to train and test part by the ratio
    
    oracle_mode: false to simulate prediction in real by 
        set the covariates of track and lap status as nan in the testset
            
    
    """    
    start = pd.Timestamp("01-01-2019", freq=freq)  # can be different for each time series

    train_set = []
    test_set = []
    
    #select run
    if runs>=0:
        _telemetry_data = [telemetry_data[runs].copy()]
    else:
        _telemetry_data = telemetry_data.copy()
    
   
    #_data: eventid, carids, datalist[carnumbers, features, tslen]->
    #[laptime, rank, track, lap]]
    #telemetry dataset <eventid, carids, ts[carid][second_seqid, vspeed, distance...])>; filled with NaN
    for _data in _telemetry_data:
        _train = []
        _test = []
        
        #statistics on the ts length
        ts_len = [ _entry.shape[0] for _entry in _data[2]]
        train_len = int(np.max(ts_len) * train_ratio)
        
        print(f'====event:{events[_data[0]]}, train_len={train_len}, max_len={np.max(ts_len)}, min_len={np.min(ts_len)}')
                
        # process for each ts
        for rowid in range(len(_data[2])):
            # teleColId['vspeed']
            # rec:= [ts_seqid, distance, vspeed...]
            rec = _data[2][rowid].copy()
            
            ##remove all nan
            #rec = rec[~np.isnan(rec)]
            # remove nan at tail
            idx = 1
            while idx < rec.shape[0]:
                nan_count = np.sum(np.isnan(rec[idx]))
                #if rec[-idx,0] == np.nan or rec[-idx,1] == np.nan or rec[-idx,2] == np.nan:
                if nan_count > 0:
                    idx += 1
                    continue
                break
            if idx > 1:
                rec = rec[:-idx]
            
            # remove short ts
            totallen = rec.shape[0]
            if ( totallen < train_len + prediction_length):
                print(f'a short ts: carid={_data[1][rowid]}，len={totallen}, nancnt={nan_count}')
                continue
                
            # nan interpolate
            _checker = rec[:, run_ts]
            nans, x= nan_helper(_checker)
            nan_count = np.sum(nans) 
            if nan_count > 0 and interpolate:
                rec[nans,run_ts]= np.interp(x(nans), x(~nans), _checker[~nans])
            
            if nan_count > nan_threshold * totallen:
                #skip this ts, if there too many nans
                print(f'carno:{_data[1][rowid]}, totallen:{totallen}, nancount:{nan_count}, skip')
                continue            

            ip_cols=[COL_TRACKSTATUS,COL_LAPSTATUS]
            for col in ip_cols:
                _checker = rec[:, col]
                nans, x= nan_helper(_checker)
                nan_count = np.sum(nans) 
                if nan_count > 0:
                    if interpolate:
                        rec[nans,col]= np.interp(x(nans), x(~nans), _checker[~nans])
                    else:
                        #force to clear
                        rec[nans,col]= 0
                
            if use_global_dict:
                carno = _data[1][rowid]
                carid = global_carids[_data[1][rowid]]
            else:
                #simulation dataset, todo, fix the carids as decoder
                carno = rowid
                carid = rowid
                
            # tranpose [features, ts]
            rec = rec.T
                
            # selection of features
            if test_flag(oracle_mode, MODE_NOTRACK):                
                rec[COL_TRACKSTATUS, :] = 0
            if test_flag(oracle_mode, MODE_NOLAP):                
                rec[COL_LAPSTATUS, :] = 0
                
            # split and add to dataset record
            _train.append({'target': rec[run_ts,:train_len].astype(np.float32), 
                            'start': start, 
                            'feat_static_cat': [carid],
                            'feat_dynamic_real': [rec[COL_TRACKSTATUS,:train_len],
                                   rec[COL_LAPSTATUS,:train_len]]
                          }
                          )
            
            # multiple test ts(rolling window as half of the prediction_length)
            test_rec_cnt = 0
            for endpos in range(totallen, train_len+prediction_length, 
                                -int(prediction_length/2)):
                
                track_rec = rec[COL_TRACKSTATUS, :endpos].copy()
                lap_rec = rec[COL_LAPSTATUS, :endpos].copy()
                
                # test mode
                if test_flag(oracle_mode, MODE_TEST):
                    #set prediction part as nan
                    #track_rec[-prediction_length:] = np.nan
                    #lap_rec[-prediction_length:] = np.nan
                    
                    # since nan does not work, use cur-val instead
                    track_rec[-prediction_length:] = track_rec[-prediction_length - 1]
                    #track_rec[-prediction_length:] = random.randint(0,1)
                    #lap_rec[-prediction_length:] = lap_rec[-prediction_length - 1]
                    lap_rec[-prediction_length:] = 0
                    
                
                _test.append({'target': rec[run_ts,:endpos].astype(np.float32), 
                            'start': start, 
                            'feat_static_cat': [carid],
                            'feat_dynamic_real': [track_rec,lap_rec]
                            #'feat_dynamic_real': [rec[COL_TRACKSTATUS,:endpos],
                            #       rec[COL_LAPSTATUS,:endpos]] 
                             }
                          )   
                test_rec_cnt += 1
            
            #add one ts
            print(f'carno:{carno}, totallen:{totallen}, nancount:{nan_count}, test_reccnt:{test_rec_cnt}')

        train_set.extend(_train)
        test_set.extend(_test)

    print(f'train len:{len(train_set)}, test len:{len(test_set)}')
    
    train_ds = ListDataset(train_set, freq=freq)
    test_ds = ListDataset(test_set, freq=freq)    
    
    return train_ds, test_ds, train_set, test_set

def save_dataset(datafile,freq, prediction_length, cardinality, train_ds, test_ds):
    with open(datafile, 'wb') as f:
        #pack [global_carids, laptime_data]
        savedata = [freq, prediction_length, cardinality, train_ds, test_ds]
        #savedata = [freq, train_set, test_set]
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)

In [30]:
def make_gluontsdb():
    #run on indy dataset
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq,interpolate=interpolate)
    save_dataset(f'vspeed-oracle{ipstr}-f{freq}-t{prediction_length}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq, oracle_mode=MODE_NOTRACK,interpolate=interpolate)
    save_dataset(f'vspeed-oracle{ipstr}-notrack-f{freq}-t{prediction_length}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq, oracle_mode=MODE_NOLAP,interpolate=interpolate)
    save_dataset(f'vspeed-oracle{ipstr}-nolap-f{freq}-t{prediction_length}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq, oracle_mode=MODE_TEST,interpolate=interpolate)
    save_dataset(f'vspeed-oracle{ipstr}-testcurtrack-f{freq}-t{prediction_length}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq, oracle_mode=MODE_TEST + MODE_NOLAP
                                         ,interpolate=interpolate)
    save_dataset(f'vspeed-oracle{ipstr}-nolapcurtrack-f{freq}-t{prediction_length}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)

In [33]:
# global configuration
prediction_length = 200
prediction_length = 60

freq = "1min"
cardinality = [len(global_carids)]
interpolate = False
ipstr = '' if interpolate else '-noip'

plens=[5,10,20,30]
for prediction_length in plens:
    make_gluontsdb()

====event:Phoenix, train_len=5004, max_len=6256, min_len=892
carno:1, totallen:6240, nancount:0, test_reccnt:616
carno:4, totallen:6256, nancount:119, test_reccnt:624
a short ts: carid=5，len=4081, nancnt=0
carno:6, totallen:6243, nancount:0, test_reccnt:617
carno:9, totallen:6244, nancount:0, test_reccnt:618
carno:10, totallen:5693, nancount:202, test_reccnt:342
a short ts: carid=12，len=3964, nancnt=0
carno:14, totallen:6250, nancount:0, test_reccnt:621
carno:15, totallen:6250, nancount:19, test_reccnt:621
carno:18, totallen:6251, nancount:22, test_reccnt:621
a short ts: carid=19，len=892, nancnt=0
carno:20, totallen:6249, nancount:121, test_reccnt:620
carno:21, totallen:6252, nancount:0, test_reccnt:622
carno:22, totallen:6250, nancount:618, test_reccnt:621
carno:23, totallen:6253, nancount:0, test_reccnt:622
carno:26, totallen:6255, nancount:12, test_reccnt:623
carno:27, totallen:6243, nancount:0, test_reccnt:617
carno:28, totallen:6244, nancount:0, test_reccnt:618
carno:30, totallen:

carno:14, totallen:6250, nancount:0, test_reccnt:621
carno:15, totallen:6250, nancount:19, test_reccnt:621
carno:18, totallen:6251, nancount:22, test_reccnt:621
a short ts: carid=19，len=892, nancnt=0
carno:20, totallen:6249, nancount:121, test_reccnt:620
carno:21, totallen:6252, nancount:0, test_reccnt:622
carno:22, totallen:6250, nancount:618, test_reccnt:621
carno:23, totallen:6253, nancount:0, test_reccnt:622
carno:26, totallen:6255, nancount:12, test_reccnt:623
carno:27, totallen:6243, nancount:0, test_reccnt:617
carno:28, totallen:6244, nancount:0, test_reccnt:618
carno:30, totallen:6251, nancount:35, test_reccnt:621
a short ts: carid=32，len=4429, nancnt=0
carno:59, totallen:6255, nancount:15, test_reccnt:623
carno:88, totallen:6254, nancount:0, test_reccnt:623
carno:98, totallen:6250, nancount:0, test_reccnt:621
====event:Indy500, train_len=8659, max_len=10824, min_len=1969
carno:1, totallen:10791, nancount:184, test_reccnt:1064
a short ts: carid=3，len=7918, nancnt=0
carno:4, tot

carno:26, totallen:6255, nancount:12, test_reccnt:623
carno:27, totallen:6243, nancount:0, test_reccnt:617
carno:28, totallen:6244, nancount:0, test_reccnt:618
carno:30, totallen:6251, nancount:35, test_reccnt:621
a short ts: carid=32，len=4429, nancnt=0
carno:59, totallen:6255, nancount:15, test_reccnt:623
carno:88, totallen:6254, nancount:0, test_reccnt:623
carno:98, totallen:6250, nancount:0, test_reccnt:621
====event:Indy500, train_len=8659, max_len=10824, min_len=1969
carno:1, totallen:10791, nancount:184, test_reccnt:1064
a short ts: carid=3，len=7918, nancnt=0
carno:4, totallen:10797, nancount:249, test_reccnt:1067
carno:6, totallen:10791, nancount:201, test_reccnt:1064
carno:7, totallen:10806, nancount:279, test_reccnt:1071
carno:9, totallen:10787, nancount:88, test_reccnt:1062
a short ts: carid=10，len=2868, nancnt=0
carno:12, totallen:10782, nancount:85, test_reccnt:1059
a short ts: carid=13，len=3557, nancnt=0
carno:14, totallen:9993, nancount:171, test_reccnt:665
carno:15, tota

carno:88, totallen:6254, nancount:0, test_reccnt:623
carno:98, totallen:6250, nancount:0, test_reccnt:621
====event:Indy500, train_len=8659, max_len=10824, min_len=1969
carno:1, totallen:10791, nancount:184, test_reccnt:1064
a short ts: carid=3，len=7918, nancnt=0
carno:4, totallen:10797, nancount:249, test_reccnt:1067
carno:6, totallen:10791, nancount:201, test_reccnt:1064
carno:7, totallen:10806, nancount:279, test_reccnt:1071
carno:9, totallen:10787, nancount:88, test_reccnt:1062
a short ts: carid=10，len=2868, nancnt=0
carno:12, totallen:10782, nancount:85, test_reccnt:1059
a short ts: carid=13，len=3557, nancnt=0
carno:14, totallen:9993, nancount:171, test_reccnt:665
carno:15, totallen:10793, nancount:193, test_reccnt:1065
carno:17, totallen:10799, nancount:525, test_reccnt:1068
a short ts: carid=18，len=6851, nancnt=0
carno:19, totallen:10789, nancount:176, test_reccnt:1063
carno:20, totallen:10785, nancount:207, test_reccnt:1061
carno:21, totallen:10799, nancount:176, test_reccnt:10

carno:6, totallen:10791, nancount:201, test_reccnt:1064
carno:7, totallen:10806, nancount:279, test_reccnt:1071
carno:9, totallen:10787, nancount:88, test_reccnt:1062
a short ts: carid=10，len=2868, nancnt=0
carno:12, totallen:10782, nancount:85, test_reccnt:1059
a short ts: carid=13，len=3557, nancnt=0
carno:14, totallen:9993, nancount:171, test_reccnt:665
carno:15, totallen:10793, nancount:193, test_reccnt:1065
carno:17, totallen:10799, nancount:525, test_reccnt:1068
a short ts: carid=18，len=6851, nancnt=0
carno:19, totallen:10789, nancount:176, test_reccnt:1063
carno:20, totallen:10785, nancount:207, test_reccnt:1061
carno:21, totallen:10799, nancount:176, test_reccnt:1068
carno:22, totallen:10789, nancount:384, test_reccnt:1063
carno:23, totallen:10824, nancount:200, test_reccnt:1080
a short ts: carid=24，len=8246, nancnt=0
carno:25, totallen:10816, nancount:210, test_reccnt:1076
carno:26, totallen:10798, nancount:259, test_reccnt:1067
carno:27, totallen:10787, nancount:194, test_recc

...

carno:27, totallen:7171, nancount:48, test_reccnt:140
a short ts: carid=28，len=5411, nancnt=0
carno:30, totallen:7172, nancount:52, test_reccnt:140
carno:59, totallen:7179, nancount:219, test_reccnt:141
carno:88, totallen:7178, nancount:48, test_reccnt:141
carno:98, totallen:7176, nancount:47, test_reccnt:141
train len:97, test len:14572
====event:Phoenix, train_len=5004, max_len=6256, min_len=892
carno:1, totallen:6240, nancount:0, test_reccnt:81
carno:4, totallen:6256, nancount:119, test_reccnt:82
a short ts: carid=5，len=4081, nancnt=0
carno:6, totallen:6243, nancount:0, test_reccnt:81
carno:9, totallen:6244, nancount:0, test_reccnt:81
carno:10, totallen:5693, nancount:202, test_reccnt:44
a short ts: carid=12，len=3964, nancnt=0
carno:14, totallen:6250, nancount:0, test_reccnt:82
carno:15, totallen:6250, nancount:19, test_reccnt:82
carno:18, totallen:6251, nancount:22, test_reccnt:82
a short ts: carid=19，len=892, nancnt=0
carno:20, totallen:6249, nancount:121, test_reccnt:81
carno:21,

carno:21, totallen:6252, nancount:0, test_reccnt:82
carno:22, totallen:6250, nancount:618, test_reccnt:82
carno:23, totallen:6253, nancount:0, test_reccnt:82
carno:26, totallen:6255, nancount:12, test_reccnt:82
carno:27, totallen:6243, nancount:0, test_reccnt:81
carno:28, totallen:6244, nancount:0, test_reccnt:81
carno:30, totallen:6251, nancount:35, test_reccnt:82
a short ts: carid=32，len=4429, nancnt=0
carno:59, totallen:6255, nancount:15, test_reccnt:82
carno:88, totallen:6254, nancount:0, test_reccnt:82
carno:98, totallen:6250, nancount:0, test_reccnt:82
====event:Indy500, train_len=8659, max_len=10824, min_len=1969
carno:1, totallen:10791, nancount:184, test_reccnt:141
a short ts: carid=3，len=7918, nancnt=0
carno:4, totallen:10797, nancount:249, test_reccnt:141
carno:6, totallen:10791, nancount:201, test_reccnt:141
carno:7, totallen:10806, nancount:279, test_reccnt:142
carno:9, totallen:10787, nancount:88, test_reccnt:140
a short ts: carid=10，len=2868, nancnt=0
carno:12, totallen:

carno:4, totallen:10797, nancount:249, test_reccnt:141
carno:6, totallen:10791, nancount:201, test_reccnt:141
carno:7, totallen:10806, nancount:279, test_reccnt:142
carno:9, totallen:10787, nancount:88, test_reccnt:140
a short ts: carid=10，len=2868, nancnt=0
carno:12, totallen:10782, nancount:85, test_reccnt:140
a short ts: carid=13，len=3557, nancnt=0
carno:14, totallen:9993, nancount:171, test_reccnt:87
carno:15, totallen:10793, nancount:193, test_reccnt:141
carno:17, totallen:10799, nancount:525, test_reccnt:141
a short ts: carid=18，len=6851, nancnt=0
carno:19, totallen:10789, nancount:176, test_reccnt:140
carno:20, totallen:10785, nancount:207, test_reccnt:140
carno:21, totallen:10799, nancount:176, test_reccnt:141
carno:22, totallen:10789, nancount:384, test_reccnt:140
carno:23, totallen:10824, nancount:200, test_reccnt:143
a short ts: carid=24，len=8246, nancnt=0
carno:25, totallen:10816, nancount:210, test_reccnt:142
carno:26, totallen:10798, nancount:259, test_reccnt:141
carno:27

carno:15, totallen:10793, nancount:193, test_reccnt:141
carno:17, totallen:10799, nancount:525, test_reccnt:141
a short ts: carid=18，len=6851, nancnt=0
carno:19, totallen:10789, nancount:176, test_reccnt:140
carno:20, totallen:10785, nancount:207, test_reccnt:140
carno:21, totallen:10799, nancount:176, test_reccnt:141
carno:22, totallen:10789, nancount:384, test_reccnt:140
carno:23, totallen:10824, nancount:200, test_reccnt:143
a short ts: carid=24，len=8246, nancnt=0
carno:25, totallen:10816, nancount:210, test_reccnt:142
carno:26, totallen:10798, nancount:259, test_reccnt:141
carno:27, totallen:10787, nancount:194, test_reccnt:140
carno:28, totallen:10789, nancount:203, test_reccnt:140
carno:29, totallen:10790, nancount:187, test_reccnt:141
a short ts: carid=30，len=1969, nancnt=0
a short ts: carid=32，len=6069, nancnt=0
a short ts: carid=33，len=2394, nancnt=0
carno:59, totallen:10785, nancount:199, test_reccnt:140
carno:60, totallen:10817, nancount:213, test_reccnt:142
carno:64, totall

carno:64, totallen:10820, nancount:192, test_reccnt:143
carno:66, totallen:10795, nancount:97, test_reccnt:141
carno:88, totallen:10797, nancount:190, test_reccnt:141
carno:98, totallen:10796, nancount:319, test_reccnt:141
====event:Texas, train_len=5822, max_len=7278, min_len=1
carno:1, totallen:7253, nancount:173, test_reccnt:94
a short ts: carid=3，len=1, nancnt=173
a short ts: carid=4，len=132, nancnt=0
carno:5, totallen:7258, nancount:7221, skip
a short ts: carid=6，len=4527, nancnt=0
a short ts: carid=7，len=1, nancnt=0
carno:9, totallen:7253, nancount:0, test_reccnt:94
carno:10, totallen:7278, nancount:0, test_reccnt:96
a short ts: carid=12，len=5791, nancnt=0
a short ts: carid=14，len=1501, nancnt=0
carno:15, totallen:7260, nancount:0, test_reccnt:94
carno:18, totallen:7262, nancount:0, test_reccnt:94
carno:19, totallen:5892, nancount:0, test_reccnt:3
a short ts: carid=20，len=4527, nancnt=0
carno:21, totallen:7270, nancount:0, test_reccnt:95
carno:22, totallen:7257, nancount:2032, sk

====event:Phoenix, train_len=5004, max_len=6256, min_len=892
carno:1, totallen:6240, nancount:0, test_reccnt:40
carno:4, totallen:6256, nancount:119, test_reccnt:40
a short ts: carid=5，len=4081, nancnt=0
carno:6, totallen:6243, nancount:0, test_reccnt:40
carno:9, totallen:6244, nancount:0, test_reccnt:40
carno:10, totallen:5693, nancount:202, test_reccnt:21
a short ts: carid=12，len=3964, nancnt=0
carno:14, totallen:6250, nancount:0, test_reccnt:40
carno:15, totallen:6250, nancount:19, test_reccnt:40
carno:18, totallen:6251, nancount:22, test_reccnt:40
a short ts: carid=19，len=892, nancnt=0
carno:20, totallen:6249, nancount:121, test_reccnt:40
carno:21, totallen:6252, nancount:0, test_reccnt:40
carno:22, totallen:6250, nancount:618, test_reccnt:40
carno:23, totallen:6253, nancount:0, test_reccnt:40
carno:26, totallen:6255, nancount:12, test_reccnt:40
carno:27, totallen:6243, nancount:0, test_reccnt:40
carno:28, totallen:6244, nancount:0, test_reccnt:40
carno:30, totallen:6251, nancount:

carno:23, totallen:10824, nancount:200, test_reccnt:71
a short ts: carid=24，len=8246, nancnt=0
carno:25, totallen:10816, nancount:210, test_reccnt:70
carno:26, totallen:10798, nancount:259, test_reccnt:70
carno:27, totallen:10787, nancount:194, test_reccnt:69
carno:28, totallen:10789, nancount:203, test_reccnt:69
carno:29, totallen:10790, nancount:187, test_reccnt:70
a short ts: carid=30，len=1969, nancnt=0
a short ts: carid=32，len=6069, nancnt=0
a short ts: carid=33，len=2394, nancnt=0
carno:59, totallen:10785, nancount:199, test_reccnt:69
carno:60, totallen:10817, nancount:213, test_reccnt:70
carno:64, totallen:10820, nancount:192, test_reccnt:71
carno:66, totallen:10795, nancount:97, test_reccnt:70
carno:88, totallen:10797, nancount:190, test_reccnt:70
carno:98, totallen:10796, nancount:319, test_reccnt:70
====event:Texas, train_len=5822, max_len=7278, min_len=1
carno:1, totallen:7253, nancount:173, test_reccnt:46
a short ts: carid=3，len=1, nancnt=173
a short ts: carid=4，len=132, nanc

carno:20, totallen:7193, nancount:47, test_reccnt:46
carno:21, totallen:7180, nancount:48, test_reccnt:46
carno:22, totallen:7173, nancount:60, test_reccnt:46
carno:23, totallen:7179, nancount:222, test_reccnt:46
carno:26, totallen:7177, nancount:48, test_reccnt:46
carno:27, totallen:7171, nancount:48, test_reccnt:46
a short ts: carid=28，len=5411, nancnt=0
carno:30, totallen:7172, nancount:52, test_reccnt:46
carno:59, totallen:7179, nancount:219, test_reccnt:46
carno:88, totallen:7178, nancount:48, test_reccnt:46
carno:98, totallen:7176, nancount:47, test_reccnt:46
train len:97, test len:4756


#### test

In [17]:
##debug data format
def _timestr(timestamp, scale=10000):
    s, ms = divmod(timestamp, scale)
    hours, remainder = divmod(s, 3600)
    minutes, seconds = divmod(remainder, 60)
    timestr = '{:02}:{:02}:{:02}.{:04}'.format(int(hours), int(minutes), int(seconds), int(ms))
    return timestr

def _gettime(timestr, scale=1000.):
    tmms = 0
    tmall = timestr.split('.')
    tms = [int(x) for x in tmall[0].split(':')]
    if len(tms) == 3 and len(tmall) == 2:
        tmms = (tms[0] * 3600 + tms[1] * 60 + tms[2])*scale + float(tmall[1])
    return tmms

def decode_cmd(cmd, idx, deli=chr(0xa6)):
    items = cmd.split(deli)
    print(str(_hex2int(items[idx])*1.0/10000))
  
def _hex2int(hexstr): 
    return int(hexstr, 16)
    

In [18]:
cmd="$C?U?200F7?R.I?7?1?2?F7?4438D18?4039D?T?3DC05?D9?0?1?B6?0?7?3DC05?Active?K?4?EE?3?0"
decode_cmd(cmd,8,'?')

7153.5896


In [19]:
cmd='$C?U?20015?R.I?3?1?2?15?748B24?3EE16?T?3E8ED?10?AE15?0?7D0A?0?3?3E8ED?Active?G?0?0?3?0'
decode_cmd(cmd,8,'?')

763.7796


In [25]:
_timestr(1345.0213)

'00:00:00.1345'

In [21]:
a = np.array([[1,2,3,4],[1,2,3,4],[1,2,3,4],[1,2,3,4]])
colidx = [1,3]
a[:,colidx]

array([[2, 4],
       [2, 4],
       [2, 4],
       [2, 4]])

In [28]:
1345.0213/60

22.417021666666667